In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sympy import im
from constants.Constants import PATH_ROUTES
from utils.CustomDataset import CustomDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from NeuralNets.CNN import CustomResNet34, ConvNetV1, ResNetV1
from helpers.customMetrics import Accuracy, AccuracyEfficient
from PIL import Image
import sys
import time


In [2]:
train_index = pd.read_csv(PATH_ROUTES.TRAIN_CSV_INDEX_PATH)

class GrayscaleToRGB:
    def __call__(self, img):
        return img.convert("RGB")
    
image_size = 256


custom_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    #GrayscaleToRGB(),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5)),
])


dataset = CustomDataset(train_index=train_index, 
                        use_cache=False, 
                        cache_max_size=10000,
                        train_size=50,
                        transform=custom_transform)

train_dataset, test_dataset = CustomDataset.split_train_test_set(dataset=dataset, test_size=0.001)

"""

train_files_path = train_dataset.file_paths
train_files_labels = train_dataset.labels

test_files_path = test_dataset.file_paths
test_files_labels = test_dataset.labels

image, label = test_dataset.__getitem__(9)
image.show()
"""

'\n\ntrain_files_path = train_dataset.file_paths\ntrain_files_labels = train_dataset.labels\n\ntest_files_path = test_dataset.file_paths\ntest_files_labels = test_dataset.labels\n\nimage, label = test_dataset.__getitem__(9)\nimage.show()\n'

In [3]:
"""
image, label = item = test_dataset.__getitem__(3)
label2 = label.view(-1, 5, 5, 3)
classes = test_dataset.classes
"""


train_dataset.__getitem__(0)





(tensor([[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -0.9922, -0.9922,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9765, -0.9765, -0.9765,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-0.9216, -0.9137, -0.9059,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9608, -0.9451, -0.9373,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -0.9922, -0.9922,  ..., -1.0000, -1.0000, -1.0000]]]),
 tensor([1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
         0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
         1., 0., 0.]))

In [4]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:

#model = CustomResNet34(5,5)
model = ConvNetV1(image_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1E-4)
device = torch.device('cuda')
model = model.to(device)

print(device)


cuda


: 

In [ ]:
num_epochs = 4
show_batch_info = True

for epoch in range(num_epochs):
    
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    current_batch = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        output_reshaped = torch.flatten(outputs, start_dim=1)
        loss = criterion(labels, output_reshaped)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            hits = Accuracy(outputs, labels)
        correct_predictions += hits
        total_predictions += labels.size(0) * labels.size(1)
        running_loss += loss.item() * inputs.size(0)
        if (show_batch_info):
            batch_info = "Batch [" + str(current_batch) + "/" + str(len(train_loader)) + "] ended"
            sys.stdout.write("\r" + batch_info)
            sys.stdout.flush()
        
        current_batch += 1

    correct_test_predictions = 0
    total_test_predictions = 0
    for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
                        
            outputs = model(inputs)
            output_reshaped = torch.flatten(outputs, start_dim=1)
            loss = criterion(labels, output_reshaped)

            hits = Accuracy(outputs, labels)
            correct_test_predictions += hits
            total_test_predictions += labels.size(0) * labels.size(1)
            
    epoch_accuracy = correct_predictions / total_predictions * 100.0
    epoch_test_accuracy = correct_test_predictions / total_test_predictions * 100.0
    epoch_loss = running_loss / len(train_loader.dataset)
    
    if (show_batch_info):
        sys.stdout.write("\r")
        sys.stdout.flush()
        sys.stdout.write("\r")
        sys.stdout.flush()
        sys.stdout.write("\r")
        sys.stdout.flush()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Test Accuracy: {epoch_test_accuracy:.2f}%')

        
        

Batch [36/117] ended

In [ ]:

SAVE_PATH = '../TrainedModels'
torch.save(model.state_dict(), SAVE_PATH)

In [ ]:
a = train_dataset.cacheDict.dict
print(len(train_dataset.cacheDict.dict))
for key in a.keys():
    print(key)

0
